# Azure Cognitive Services "Brain Tumor Detection" - Test

This notebook aims to test a brain tumor detector built using Azure Cognitive Services.
<br />
The training dataset is the same one used for building the [YOLOv5 version](https://github.com/giuseppebrb/BrainTumorDetection/blob/main/Brain_Tumor_Detector.ipynb) of this model.

In [1]:
# Installing the library needed to interact with Azure Cognitive Services
%pip install azure-cognitiveservices-vision-customvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 291 kB/s 
     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 178 kB 47.8 MB/s 
     |████████████████████████████████| 41 kB 412 kB/s 


In [2]:
# Global imports

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid
import urllib.request
import zipfile
import csv

In [3]:
# Costants and secrets
ENDPOINT = "<REPLACE_WITH_AZURE_ENDPOINT>"
prediction_key = "<REPLACE_WITH_PREDICTION_KEY>"
prediction_resource_id = "<REPLACE_WITH_RESOURCE_ID>"
project_id = "<REPLACE_WITH_PROJECT_ID>"
publish_iteration_name = "Brain Tumour Detection 1.0.0"

TAGS = ['Tumor', 'Healthy']

In [4]:
# Setting up the predictor
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [5]:
# Download and unzip the dataset
if not os.path.isfile('data.zip'):
  urllib.request.urlretrieve("https://github.com/giuseppebrb/BrainTumorDetection/blob/main/data.zip?raw=true", "data.zip")

with zipfile.ZipFile("data.zip","r") as zip_ref:
    zip_ref.extractall(".")

os.remove('data.zip')

In [6]:
# Iterates images in the specified folder and predicts using deployed model on Azure
def predict_from_folder(folder, label_folder):
    test_results = []
    for filename in os.listdir(folder):
      image_row = [None] * 5
      image_row[0] = filename
      with open(folder + filename, "rb") as image_contents:
        try:
          results = predictor.detect_image(
              project_id, publish_iteration_name, image_contents.read())
          # Display the results.
          prediction = results.predictions[0]

          image_row[1] = prediction.tag_name
          image_row[2] = "{:.2f}".format(prediction.probability)
          image_row[3] = "{:.2f}, {:.2f}, {:.2f}, {:.2f}".format(prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height)
        except:
          image_row[1] = 'Error'
          image_row[2] = 'Error'
          image_row[3] = 'Error'

        try:
          with open(label_folder + str(filename.split('.', 1)[0])+'.txt') as f:
            actual_annotation = f.readline()
            formatted_bounding_box = actual_annotation[2:]
            formatted_bounding_box = formatted_bounding_box.replace(' ', ', ', 3)
            image_row[4] = formatted_bounding_box if int(actual_annotation[0]) == 1 else TAGS[1]
        except:
          image_row[4] = 'Actual label not found'
      print(image_row)
      test_results.append(image_row)
    return test_results

In [7]:
# Folders containing test images
test_folder = "data/axial/images/validation/"
label_folder = "data/axial/labels/validation/"

# Predict and store results in a csv file
with open('test_results.csv', 'w') as f:
  writer = csv.writer(f)
  writer.writerow(['File', 'Tag', 'Probability', 'Bounding Box [left, top, width, height]', 'Actual Bounding Box [x_center, y_center, width, height]'])

  cancer_results = predict_from_folder(test_folder, label_folder)
  
  writer.writerows(cancer_results)
  

print('Test complete -> check test_results.csv')

0.649648, 0.596244, 0.083333, 0.091549
['00090_169.jpg', 'Tumor', '1.00', '0.60, 0.54, 0.10, 0.11', 'Healthy']
0.341158, 0.494131, 0.226917, 0.215962

['00322_121.jpg', 'Tumor', '1.00', '0.21, 0.39, 0.25, 0.22', '0.341158, 0.494131, 0.226917, 0.215962\n']
0.406103, 0.670188, 0.209703, 0.157277
['00334_108.jpg', 'Tumor', '1.00', '0.28, 0.58, 0.25, 0.17', '0.406103, 0.670188, 0.209703, 0.157277']
0.446792, 0.339202, 0.143975, 0.136150
['00350_135.jpg', 'Tumor', '1.00', '0.36, 0.26, 0.16, 0.15', '0.446792, 0.339202, 0.143975, 0.136150']
0.583920, 0.418427, 0.062207, 0.071596
['00053_277.jpg', 'Tumor', '1.00', '0.55, 0.38, 0.07, 0.08', 'Healthy']
0.614241, 0.536385, 0.156495, 0.183099
['00322_147.jpg', 'Tumor', '1.00', '0.52, 0.44, 0.18, 0.18', '0.614241, 0.536385, 0.156495, 0.183099']
0.354492, 0.566406, 0.150391, 0.183594

['Cancer (20).jpg', 'Tumor', '1.00', '0.30, 0.47, 0.15, 0.17', '0.354492, 0.566406, 0.150391, 0.183594\n']
0.567383, 0.527344, 0.111328, 0.144531

['Cancer (23).jpg', 